In [109]:
import pandas as pd
from pybaseball import statcast_pitcher, playerid_lookup, pitching_stats
from datetime import date
from features.mlb_features import aggregate_pitcher_games
from features.enrichments import add_opponent_k_pct, add_park_factor
from features.rolling import add_rolling_features

from pybaseball import cache
cache.enable()

In [110]:
playerid_lookup('cole', 'gerrit')

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,cole,gerrit,543037,coleg001,colege01,13125,2013.0,2024.0


In [111]:
gerrit_id = 543037

df = statcast_pitcher('2023-04-01', '2023-10-01', player_id=gerrit_id)
print(df.columns.tolist())

['pitch_type', 'game_date', 'release_speed', 'release_pos_x', 'release_pos_z', 'player_name', 'batter', 'pitcher', 'events', 'description', 'spin_dir', 'spin_rate_deprecated', 'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des', 'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type', 'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y', 'tfs_deprecated', 'tfs_zulu_deprecated', 'umpire', 'sv_id', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot', 'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'game_pk', 'fielder_2', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y', 'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle', 'woba_value', 'woba_denom', 'babip_value', 'iso_value', 'launch_speed_a

In [112]:
from pybaseball import team_batting
batting = team_batting(2023)
batting = batting[['Team', 'SO', 'PA']]
batting['K_pct'] = batting['SO'] / batting['PA']

In [113]:
# Normalize team abbreviations
batting['Team'] = batting['Team'].replace({
    'TBR': 'TB',
    'SDP': 'SD',
    'KCR': 'KC',
    'SFG': 'SF',
    'CHW': 'CWS'
})

games = aggregate_pitcher_games(df)
games = add_opponent_k_pct(games, batting)

In [114]:
pf_df = pd.read_csv("../data/raw/fangraphs_park_factors_2023.csv")
pf_df['K_park_factor'] = pf_df['SO'] / 100

team_map = {
    'Diamondbacks': 'ARI',
    'Braves': 'ATL',
    'Orioles': 'BAL',
    'Red Sox': 'BOS',
    'Cubs': 'CHC',
    'White Sox': 'CWS',
    'Reds': 'CIN',
    'Guardians': 'CLE',
    'Rockies': 'COL',
    'Tigers': 'DET',
    'Astros': 'HOU',
    'Royals': 'KC',
    'Angels': 'LAA',
    'Dodgers': 'LAD',
    'Marlins': 'MIA',
    'Brewers': 'MIL',
    'Twins': 'MIN',
    'Mets': 'NYM',
    'Yankees': 'NYY',
    'Athletics': 'OAK',
    'Phillies': 'PHI',
    'Pirates': 'PIT',
    'Padres': 'SD',
    'Giants': 'SF',
    'Mariners': 'SEA',
    'Cardinals': 'STL',
    'Rays': 'TB',
    'Rangers': 'TEX',
    'Blue Jays': 'TOR',
    'Nationals': 'WSN'
}

pf_df['Team_abbr'] = pf_df['Team'].map(team_map)
k_factors = pf_df[['Team_abbr', 'K_park_factor']].dropna()

In [115]:
games = add_park_factor(games, k_factors)
games = add_rolling_features(games)
games.head(20)

,game_date,pitch_count,strikeouts,max_inning,num_pitch_types,home_team,away_team,rest_days,pitcher_team,opponent_team,SO,PA,opponent_k_pct,park_factor_K,rolling_K_avg_3,rolling_K_avg_5,rolling_pitch_count_5,rolling_K_rate
0,2023-04-05,103,8,7,4,NYY,PHI,6,NYY,PHI,1481,6207,0.238602,1.01,5.000000,5.0,85.0,0.055000
1,2023-04-11,97,3,7,5,CLE,NYY,6,NYY,CLE,1142,6096,0.187336,1.01,5.000000,5.0,85.0,0.055000
2,2023-04-16,109,10,9,4,NYY,MIN,5,NYY,MIN,1654,6219,0.265959,1.01,5.000000,5.0,85.0,0.055000
3,2023-04-22,96,4,6,4,NYY,TOR,6,NYY,TOR,1303,6220,0.209486,1.01,7.000000,5.0,85.0,0.055000
4,2023-04-27,92,8,7,4,TEX,NYY,5,NYY,TEX,1416,6306,0.224548,1.01,5.666667,5.0,85.0,0.055000
5,2023-05-02,108,8,6,4,NYY,CLE,5,NYY,CLE,1142,6096,0.187336,1.01,7.333333,6.6,99.4,0.044266
6,2023-05-07,95,5,6,4,TB,NYY,5,NYY,TB,1420,6164,0.230370,1.02,6.666667,6.6,100.4,0.039841
7,2023-05-12,95,4,5,4,NYY,TB,5,NYY,TB,1420,6164,0.230370,1.01,7.000000,7.0,100.0,0.042000
8,2023-05-17,104,6,7,5,TOR,NYY,5,NYY,TOR,1303,6220,0.209486,1.00,5.666667,5.8,97.2,0.034979
9,2023-05-23,99,2,6,5,NYY,BAL,6,NYY,BAL,1370,6123,0.223747,1.01,5.000000,6.2,98.8,0.030364


In [116]:
pitching_stats = pitching_stats(2023)
pitching_stats.head(1)

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,...,Pit+ FC,Stf+ FS,Loc+ FS,Pit+ FS,Stuff+,Location+,Pitching+,Stf+ FO,Loc+ FO,Pit+ FO
19,10310,2023,Zack Wheeler,PHI,33,13,6,5.9,3.61,32,...,106.0,NaN,NaN,NaN,117,106,122,NaN,NaN,NaN
